In [1]:
import pandas as pd
import polars as pl
import numpy as np
import scipy.sparse as sp

from tqdm import tqdm
from typing import List, Any

## Читаем датасет

In [2]:
data = pl.read_parquet('train.parquet')
data

user_id,artist_id
str,str
"""d705b538-1bd8-…","""69c71d72-7ed8-…"
"""d705b538-1bd8-…","""30bf469f-9abd-…"
"""d705b538-1bd8-…","""a26c9335-2459-…"
"""d705b538-1bd8-…","""69c903b5-dff0-…"
"""d705b538-1bd8-…","""af8eef9d-13aa-…"
"""d705b538-1bd8-…","""293a86ee-6ce7-…"
"""d705b538-1bd8-…","""348f4909-1c48-…"
"""d705b538-1bd8-…","""ad2bf122-726e-…"
"""d705b538-1bd8-…","""cc97fc57-30b5-…"


## Метрики

В этом задании нашей задачей будем оптимизация метрик ndcg@20. Тем не менее, такая метрику сложно интерпретировать и поэтому вам также будет доступно значение метрики hitrate@20, с которой вы уже познакомились в прошлой домашке

In [3]:
TOP_K = 20


def user_hitrate(y_relevant: List[str], y_recs: List[str], k: int = TOP_K) -> int:
    return int(len(set(y_relevant).intersection(y_recs[:k])) > 0)

def user_ndcg(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: ndcg metric for user recommendations
    """
    dcg = sum([1. / np.log2(idx + 2) for idx, item in enumerate(y_rec[:k]) if item in y_rel])
    idcg = sum([1. / np.log2(idx + 2) for idx, _ in enumerate(zip(y_rel, np.arange(k)))])
    return dcg / idcg

В этом датасете идентификаторы представлены в виде строк, но для работы с ними может быть проще сделать преобразование в числа (например, для алгоритмов матричной факторизации)

In [4]:
user_mapping = {k: v for v, k in enumerate(data['user_id'].unique())}
user_mapping_inverse = {k: v for v, k in user_mapping.items()}

artist_mapping = {k: v for v, k in enumerate(data['artist_id'].unique())}
artist_mapping_inverse = {k: v for v, k in artist_mapping.items()}

In [7]:
grouped_df_with_inds = (
    data
    .with_columns([
        pl.col('user_id').apply(user_mapping.get),
        pl.col('artist_id').apply(artist_mapping.get),
    ])
    # для каждого пользователя оставим последние 3 объекта в качестве тестовой выборки,
    # а остальное будем использовать для тренировки
    .groupby('user_id')
    .agg([
        pl.col('artist_id').apply(lambda x: x[:-3]).alias('train_item_ids'),
        pl.col('artist_id').apply(lambda x: x[-3:]).alias('test_item_ids'),
    ])
)

grouped_df_with_inds

user_id,train_item_ids,test_item_ids
i64,list[i64],list[i64]
24672,"[31304, 60192, … 68156]","[25665, 14965, 87036]"
9288,"[20954, 87865, … 79468]","[6693, 83925, 85054]"
40352,"[28366, 38213, … 15083]","[64696, 56969, 22674]"
14288,"[44285, 60064, … 87421]","[40090, 63345, 73898]"
18624,"[43769, 65473, … 33404]","[81116, 7452, 21269]"
45792,"[17436, 39069, … 45418]","[51666, 79758, 542]"
35152,"[13508, 46647, … 7307]","[70023, 23129, 52791]"
23184,"[72892, 43728, … 48568]","[77528, 11400, 538]"
40816,"[65434, 7995, … 76385]","[50366, 10960, 61743]"


In [8]:
median_seq_len = int(grouped_df_with_inds['train_item_ids'].apply(len).median())
print(f"средняя длина сессии {median_seq_len}")

средняя длина сессии 42


In [9]:
# соберем строчки для разреженной матрицы
rows = []
cols = []
values = []
for user_id, train_ids, _ in grouped_df_with_inds.rows():
    rows.extend([user_id] * len(train_ids))
    values.extend([1] * len(train_ids))
    cols.extend(train_ids)

user_item_data = sp.csr_matrix((values, (rows, cols)))

## Бейзлайны

В качестве простого бейзлайна будем рекомендовать самый популярных артистов

Мы хотим сначала провалидировать такое решение, а значит в качестве популярных артистов мы возьмем только тех, кто чаще встречается в `train_item_ids`

In [29]:
top_artists = (
    grouped_df_with_inds
    .select(pl.col('train_item_ids').alias('artist_id'))
    .explode('artist_id')
    .groupby('artist_id')
    .count()
    .sort('count', descending=True)
    .head(TOP_K + median_seq_len)
)['artist_id'].to_list()

In [31]:
ndcg_list = []
hitrate_list = []

for user_id, user_history, y_rel in grouped_df_with_inds.rows():
    y_rec = top_artists.copy()
    
    ndcg_list.append(user_ndcg(y_rel, y_rec))
    hitrate_list.append(user_hitrate(y_rel, y_rec))
    
print(f'NDCG@{TOP_K} = {np.mean(ndcg_list):.5f}, Hitrate@{TOP_K} = {np.mean(hitrate_list):.5f}')

NDCG@20 = 0.01401, Hitrate@20 = 0.10248


Не забываем про фильтрацию просмотренного (для разных доменов и подходов это не всегда улучши рекомендации, но в данном случае дало прирост)

In [30]:
ndcg_list = []
hitrate_list = []

for user_id, user_history, y_rel in grouped_df_with_inds.rows():
    y_rec = [artist_id for artist_id in top_artists if artist_id not in user_history]
    
    ndcg_list.append(user_ndcg(y_rel, y_rec))
    hitrate_list.append(user_hitrate(y_rel, y_rec))
    
print(f'NDCG@{TOP_K} = {np.mean(ndcg_list):.5f}, Hitrate@{TOP_K} = {np.mean(hitrate_list):.5f}')

NDCG@20 = 0.01740, Hitrate@20 = 0.11684


## Построим файл с рекомендациями

Для построения рекомендаций теперь можем учесть все возможные данные

In [32]:
top_artists = (
    data
    .groupby('artist_id')
    .count()
    .sort('count', descending=True)
    .head(TOP_K + median_seq_len)
)['artist_id'].to_list()

In [38]:
submission = []

for user_id, user_history in data.groupby('user_id').agg(pl.col('artist_id')).rows():
    y_rec = top_artists.copy()
    
    submission.append((user_id, y_rec))
    
submission = pl.DataFrame(submission, schema=('user_id', 'y_rec'))
submission.write_parquet('sample_submission.parquet')
submission

user_id,y_rec
str,list[str]
"""30d49a86-6e30-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""ca624347-c611-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""58dcd8a1-9c0b-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""dcc19eaf-38b0-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""14f169f8-ff3a-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""1502da3b-4c78-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""e6776ed0-5ab3-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""37d7999c-301d-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""fe182f57-8eed-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"


Не забывайте, что в файле с рекомендациями должны быть **исходные идентификаторы (строки)**, а не преобразованные в числа!